In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup, CData
import requests

from time import sleep
import pandas as pd 
import numpy as np

In [278]:
driver = webdriver.Chrome("/Applications/chromedriver")

In [96]:
# get main shelter that sit on top of parent shelters => Main > Parent > Child

driver.get("https://www.carrefour.fr/r?noRedirect=0")

page = driver.find_element_by_class_name('search-sidenav-item__list')
parent_urls = [elem.get_attribute("href") for elem in page.find_elements_by_tag_name('a')]
parent_urls = parent_urls[3:14]

urls = []

for children in parent_urls:
    driver.get(children)
    page = driver.find_element_by_class_name('search-sidenav-item--opened').find_element_by_class_name('search-sidenav-item__list')
    children_urls = [elem.get_attribute("href") for elem in page.find_elements_by_tag_name('a')]

    for child in children_urls:
        driver.get(child)
        try:
            page = driver.find_element_by_class_name('search-sidenav-item--opened').find_element_by_class_name('search-sidenav-item__list')
            a = [elem.get_attribute("href") for elem in page.find_elements_by_tag_name('a')]
            for url in a:
                urls.append(url)
        except:
            next
        
        
urls

['https://www.carrefour.fr/r/fruits-et-legumes/fruits/fruits-de-saison',
 'https://www.carrefour.fr/r/fruits-et-legumes/fruits/agrumes',
 'https://www.carrefour.fr/r/fruits-et-legumes/fruits/pommes-poires-et-raisins',
 'https://www.carrefour.fr/r/fruits-et-legumes/fruits/bananes-et-kiwis',
 'https://www.carrefour.fr/r/fruits-et-legumes/fruits/fruits-exotiques',
 'https://www.carrefour.fr/r/fruits-et-legumes/fruits/fruits-rouges',
 'https://www.carrefour.fr/r/fruits-et-legumes/fruits/melons-et-pasteques',
 'https://www.carrefour.fr/r/fruits-et-legumes/fruits/prunes-peches-nectarines-abricots',
 'https://www.carrefour.fr/r/fruits-et-legumes/legumes/legumes-de-saison',
 'https://www.carrefour.fr/r/fruits-et-legumes/legumes/choux-poireaux-navets-et-celeris',
 'https://www.carrefour.fr/r/fruits-et-legumes/legumes/carottes-et-pommes-de-terre',
 'https://www.carrefour.fr/r/fruits-et-legumes/legumes/salades-et-tomates',
 'https://www.carrefour.fr/r/fruits-et-legumes/legumes/concombres-avocats-

In [279]:
product_pages = []

headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.70 Safari/537.36'}

for link in urls:
    
    driver.get(link)

    sleep(1)
    shelter_main = driver.find_element_by_class_name('breadcrumb-trail__list').find_elements_by_css_selector('li')[-3].text
    shelter_parent = driver.find_element_by_class_name('breadcrumb-trail__list').find_elements_by_css_selector('li')[-2].text
    shelter_child = driver.find_element_by_class_name('breadcrumb-trail__list').find_elements_by_css_selector('li')[-1].text

    try:
        actions = ActionChains(driver)
        actions.move_to_element(driver.find_elements_by_css_selector("article")[-1]).perform()
    except:
        continue

    request_text = requests.get(link, headers=headers)   
    soup = BeautifulSoup(request_text.text,"html.parser" ,from_encoding="utf-8")  

    brands = []
    try:
        for label in driver.find_element_by_id('data_facet_marque').find_elements_by_css_selector("label"):
            brands.append(label.find_element_by_css_selector("span").text)
    except:
        continue

    product_pages.append({"url":link, 
                             "shelter_main": shelter_main,
                             "shelter_parent": shelter_parent,
                             "shelter_child": shelter_child,
                             "brands":brands,
                             "soup": soup.select('article')
                            })



JavascriptException: Message: javascript error: Cannot read property 'left' of undefined
  (Session info: chrome=78.0.3904.70)


In [280]:
data = pd.DataFrame(product_pages)
#data.to_csv('carrefour_catalog.csv')


In [284]:
pages = []
for index, row in data.iterrows():
    soup = row.soup
    pages.append({"shelter_main": row.shelter_main,
                  "shelter_parent": row.shelter_parent,
                  "shelter_child": row.shelter_child,                  
                  "brands": row.brands,
                  "page": soup
                })

In [325]:
pages[0]["page"][1]

<article class="ds-product-card ds-product-card--vertical">
<div>
<div class="ds-product-card--vertical-franceflag">
<div class="product-card-franceflag"></div>
</div>
<div class="ds-product-card--vertical-icons">
<div class="add-to-shoppinglist" ean="3276550243588" is="add-to-shoppinglist"></div>
</div>
<div class="ds-product-card--vertical-highlightlabel">
<div class="product-card-highlightlabel product-card-highlightlabel--fid">
<img alt="Prime Bio" class="image product-card-highlightlabel__club-icon" src="/images/badges/flag-club-bio.svg"/>
<span class="ds-body-text ds-body-text--size-s ds-body-text--color-inherit ds-body-text--weight-bold">
  Prime Bio
</span>
</div>
</div>
<div class="ds-product-card--vertical-infos">
<div class="ds-product-card--vertical-text">
<a class="product-card-title product-card-title--food" href="/p/poire-bio-williams-carrefour-3276550243588?t=R01F01SF01" rel="" target="" title="Poire bio Williams CARREFOUR">
<h2 class="ds-title ds-title--s ds-title--med

In [331]:
products = []
for page in pages:
    for content in page['page'][1:-1]:
        name = content.find("a", class_="product-card-title--food")["title"]
        packaging = content.find("div", class_="product-card-packaging").text.strip()
        price = content.find("span", class_="product-card-price__price--final").text.strip()
        price_per_unit = content.find("div", class_="product-card-per-unit-label").text.strip()
#           promo_price = line[14:-1]
        image_url = content.find("img", class_="product-card-image__image")["src"]
        origin = content.find("div", class_="product-card__origin").text.strip() if content.find("div", class_="product-card__origin") else ""
        url = content.find("a", class_="product-card-title--food")["href"]
        product_id = content.find("div", class_="add-to-shoppinglist")["ean"]
        offer = content.find("div", class_="ds-product-card--vertical-highlightlabel").text.strip()
        offer_description = content.find("span", class_="promotion-label__txt").text.strip() if content.find("span", class_="promotion-label__txt") else ""
        if len(page['brands']) > 0:
            brand = ''.join([brand for brand in page['brands'] if brand in name])
        else:
            brand = ""
                
        products.append({"shelter_main": page['shelter_main'],
                         "shelter_parent": page['shelter_parent'],
                         "shelter_child": page['shelter_child'],  
                         "name":name, 
                         "packaging":packaging,                          
                         "brand": brand,
                         "price": price,
                         "price_per_unit":price_per_unit,
                        # "promo_price": promo_price,
                         "origin": origin,
                         "image_url": image_url,                    
                         "url": url,
                         "product_id": product_id,
                         "offer": offer,
                         "offer_description": offer_description
                        })

In [332]:
product_data = pd.DataFrame(products)

export_csv = product_data.to_csv (r'/Users/vincentsalamand/Downloads/products_carrefour.csv', index = None, header=True)


In [333]:
product_data.head()

,brand,image_url,name,offer,offer_description,origin,packaging,price,price_per_unit,product_id,shelter_child,shelter_main,shelter_parent,url
0,CARREFOUR,/media/280x280/Photosite/PRODUITS_FRAIS_TRANSF...,Poire bio Williams CARREFOUR,Prime Bio,*Primes Fidélité: -10% si inscrit,FRANCE,le lot de 4,"2,95€",2.95 € / Kilogramme,3276550243588,Fruits de saison,Fruits et Légumes,Fruits,/p/poire-bio-williams-carrefour-3276550243588?...
1,CARREFOUR,/media/280x280/Photosite/PRODUITS_FRAIS_TRANSF...,Poires Guyot vertes bio CARREFOUR BIO,Prime Bio,*Primes Fidélité: -10% si inscrit,FRANCE,la barquette de 4 fruits,"2,75€",2.75 € / Pièce,3276550104971,Fruits de saison,Fruits et Légumes,Fruits,/p/poires-guyot-vertes-bio-carrefour-bio-32765...
2,CARREFOUR,/media/280x280/Photosite/PRODUITS_FRAIS_TRANSF...,Pommes Gala bio CARREFOUR BIO,Prime Bio,*Primes Fidélité: -10% si inscrit,FRANCE,la barquette de 6 fruits,"3,55€",3.55 € / Pièce,3276550063650,Fruits de saison,Fruits et Légumes,Fruits,/p/pommes-gala-bio-carrefour-bio-3276550063650...
3,,/media/280x280/Photosite/PRODUITS_FRAIS_TRANSF...,Pommes Reinette grise du Canada bio,Prime Bio,*Primes Fidélité: -10% si inscrit,FRANCE,la barquette de 4 fruits - 500gr,"2,80€",5.60 € / Kilogramme,3276550310464,Fruits de saison,Fruits et Légumes,Fruits,/p/pommes-reinette-grise-du-canada-bio-3276550...
4,CARREFOUR,/media/280x280/Photosite/PRODUITS_FRAIS_TRANSF...,Pommes Gala FILIERE QUALITE CARREFOUR,Prix imbattable !,,FRANCE,le lot de 6 - 1 kg,"1,99€",1.99 € / Kilogramme,3276552249762,Fruits de saison,Fruits et Légumes,Fruits,/p/pommes-gala-filiere-qualite-carrefour-32765...
